In [2]:
import pandas as pd

In [8]:
df = pd.read_excel("данные для обработки.xlsx")
df.head()

,Пол,Образование,Уровень должности,Настороженность,Расчётливость,Жёсткость,Независимость,Авантюризм,Непрактичность,Осторожность,Нормативность,Жертвенность,Эмоциональность,Расслабленность,Безынициативность
0,1,3,3,51,43,17,91,40,21,28,35,32,91,30,33
1,1,4,6,96,63,94,81,22,93,59,35,19,19,18,15
2,2,4,5,84,26,87,26,22,36,34,57,45,30,27,80
3,1,4,5,94,53,81,77,2,49,20,19,47,2,85,44
4,1,3,6,79,4,87,89,22,13,74,30,59,56,82,15


In [12]:
df['Пол'].value_counts()

Пол
1       691
2       342
1, 1      1
Name: count, dtype: int64

In [13]:
df['Образование'].value_counts()

Образование
3       731
4       214
2        76
1        12
3, 3      1
Name: count, dtype: int64

In [14]:
df['Уровень должности'].value_counts()

Уровень должности
2    334
3    245
4    121
5    118
6     96
1     76
7     23
8     21
Name: count, dtype: int64

In [30]:
res = df[df['Уровень должности'] > 6]
res['Пол'].value_counts()

Пол
1    35
2     9
Name: count, dtype: int64

In [32]:
res['Образование'].value_counts()

Образование
3    24
4    13
2     7
Name: count, dtype: int64